## In this study, we implemented three different models, retro-causal (lstm stateful = true and stateful=false), causal (lstm stateful = true and stateful = false), and causal-retro-causal (lstm bi-direction) using tensor flow 2.x with keras to predict the future prices of the Chicago Copper May 20 (HGK20.CMX) dataset. The dataset contains 1162 enteries and six columns. We used the column labelled close as our predictive variables to make our predictions. After the implementation of the three models and compared their performances, we realised that the causal-retro-causal (lstm bi-directional) outperformed the others, and then, the causal model showing that the retro-causal has the poorest performance here. Therefore, we can conclude that causal-retro-causal (bi-direction) parameter choice model perform better in time series forecasting since it includes the information flows in both causal and retro-causal models. 

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow.keras import layers

In [0]:
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras import metrics
from keras.layers import Dense, Dropout, LSTM
from sklearn.preprocessing import MinMaxScaler
import math
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tensorflow.keras import regularizers

In [0]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=12):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return np.array(dataX), np.array(dataY)
# fix random seed for reproducibility
np.random.seed(7)
# load the dataset
dataframe = pd.read_csv('HGK20.CMX.csv', usecols=[1], engine='python')
dataset = dataframe.values
dataset = dataset.astype('float32')
# normalize the dataset with min-max
#sc = MinMaxScaler(feature_range=(0, 1))
#data = sc.fit_transform(dataset)

In [0]:
# normalise the dataset with standardisation
sc = StandardScaler()
data = sc.fit_transform(dataset)

In [0]:
# checking the structure of the dataset
data

In [0]:
# split into train and test sets
train_size = int(len(data) * 0.7)
test_size = len(data) - train_size
train, test = data[0:train_size,:], data[train_size:len(data),:]

In [0]:
# reshape into X=t and Y=t+1
look_back = 12
X_train, y_train = create_dataset(train, look_back)
X_test, Y_test = create_dataset(test, look_back)

In [0]:
print("trainX shape: ",X_train.shape)
print("trainY shape: ",y_train.shape)
print("testX shape: ",X_test.shape)
print("testY shape: ",Y_test.shape)

In [0]:
# reshape input to be [samples, time steps, features]
X_train = np.reshape(X_train,(X_train.shape[0],X_train.shape[1],1))
X_test = np.reshape(X_test,(X_test.shape[0], X_test.shape[1], 1))

In [0]:
print("trainX shape: ",X_train.shape)
print("trainY shape: ",y_train.shape)
print("testX shape: ",X_test.shape)
print("testY shape: ",Y_test.shape)

In [0]:
tf.compat.v1.reset_default_graph()

## Causal (stateful = true) LSTM Model

In [0]:
model_stateful = tf.keras.Sequential([
tf.keras.layers.LSTM(64,  batch_input_shape=(1, 12, 1), stateful=True),
tf.keras.layers.Dense(1)])

In [0]:
model_stateful.compile(loss = 'mean_squared_error', optimizer='adam')

In [0]:
model_stateful.summary()

In [0]:
# training with the model
history = model_stateful.fit(X_train, y_train, epochs=100, batch_size=1, verbose = 1, validation_data=(X_test, Y_test), shuffle=False)
model_stateful.reset_states()

In [0]:
plt.figure(figsize=(10, 6))
#plt.plot(history.history['val_loss'], color='red', label='validation loss')
plt.plot(history.history['loss'], color='blue', label='train loss')
plt.legend(fontsize=15)
plt.title('Causal stateful = true Model Loss')
plt.ylabel('Mean Squared Error')
plt.xlabel('Epoch')
plt.show()

In [0]:
batch_size=1
train_Score = model_stateful.evaluate(X_train, y_train, batch_size=batch_size, verbose=0)
model_stateful.reset_states()
train_Score = math.sqrt(train_Score)
train_Score = sc.inverse_transform(np.array([[train_Score]]))

In [0]:
test_Score = model_stateful.evaluate(X_test, Y_test, batch_size=1, verbose=0)
model_stateful.reset_states()
test_Score = math.sqrt(test_Score)
test_Score = sc.inverse_transform(np.array([[test_Score]]))

In [0]:
print('This is the train_Score:',train_Score)
print('This is the test_Score:', test_Score)

In [0]:
train_Predict = model_stateful.predict(X_train, batch_size=1)
test_Predict = model_stateful.predict(X_test, batch_size=1)

In [0]:
mse = mean_squared_error(Y_test, test_Predict)
RMSE = np.sqrt(mse)
mae = mean_absolute_error(Y_test, test_Predict)

In [0]:
print('The Mean Squared Error is: ', mse)
print
print('The Root Mean Squared Error is :', RMSE)
print
print('The Mean Absolute Error is ', mae)

In [0]:
test_Predict

In [0]:
# Shifting train predictions for plotting
train_Plot = np.empty_like(data)
train_Plot[:, :] = np.nan
train_Plot[look_back:len(train_Predict)+look_back, :] = train_Predict

In [0]:
# Shifting test predictions for plotting
test_Plot = np.empty_like(data)
test_Plot[:, :] = np.nan
test_Plot[len(train_Predict)+(look_back*2)+1:len(data)-1, :] = test_Predict

In [0]:
plt.figure(figsize=(10, 6))
plt.plot(Y_test, color='red', label='actual data')
plt.plot(test_Predict, color='blue', label='test data')
plt.xlabel('Time [Days]', fontsize=15)
plt.ylabel('Stock Values', fontsize=15)
plt.title(' Model Real values vs Prediction values', fontsize=15)
plt.legend(fontsize=15)
plt.show()

In [0]:
plt.figure(figsize=(10, 6))
plt.plot(data, color='red', label='actual data')
plt.plot(train_Plot, color='blue', linestyle="-", label='train data')
plt.plot(test_Plot, color='green', linestyle="-", label='test data')
plt.xlabel('Time [Days]', fontsize=15)
plt.ylabel('Stock Values', fontsize=15)
plt.title('Baseline Model Prediction on HGK20.CXM',fontsize=15)
plt.legend(fontsize=15)
plt.show()

## Causal (stateful = false) LSTM Model

In [0]:
model_stateless = tf.keras.Sequential([
tf.keras.layers.LSTM(64, input_shape=(6, 1), stateful=False),
tf.keras.layers.Dense(1)])

In [0]:
model_stateless.compile(loss = 'mean_squared_error', optimizer='adam')
model_stateless.summary()

In [0]:
history = model_stateless.fit(X_train, y_train, epochs=100, batch_size=1, verbose = 1, validation_data=(X_test, Y_test), shuffle=False)

In [0]:
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], color='blue', label='train loss')
plt.title('Causal stateful = false Model Loss')
plt.ylabel('Mean Squared Error')
plt.xlabel('Epoch')
plt.show()

In [0]:
batch_size=1
train_Score = model_stateless.evaluate(X_train, y_train, batch_size=batch_size, verbose=0)
model_stateless.reset_states()
train_Score = math.sqrt(train_Score)
train_Score = sc.inverse_transform(np.array([[train_Score]]))

In [0]:
test_Score = model_stateless.evaluate(X_test, Y_test, batch_size=batch_size, verbose=0)
model_stateless.reset_states()
test_Score = math.sqrt(test_Score)
test_Score = sc.inverse_transform(np.array([[test_Score]]))

In [0]:
print('This is the train_Score:',train_Score)
print('This is the test_Score:', test_Score)

In [0]:
# generate predictions for training and testing
train_Predict1 = model_stateless.predict(X_train, batch_size=1)
test_Predict1 = model_stateless.predict(X_test, batch_size=1)

In [0]:
mse = mean_squared_error(Y_test, test_Predict1)
RMSE = np.sqrt(mse)
mae = mean_absolute_error(Y_test, test_Predict1)


In [0]:
print('The Mean Squared Error is: ', mse)
print
print('The Root Mean Squared Error is :', RMSE)
print
print('The Mean Absolute Error is ', mae)

In [0]:
test_Predict1

In [0]:
# Shifting train predictions for plotting
train_Plot_1 = np.empty_like(data)
train_Plot_1[:, :] = np.nan
train_Plot_1[look_back:len(train_Predict1)+look_back, :] = train_Predict1

In [0]:
# Shifting test predictions for plotting
test_Plot_1 = np.empty_like(data)
test_Plot_1[:, :] = np.nan
test_Plot_1[len(train_Predict1)+(look_back*2)+1:len(data)-1, :] = test_Predict1

In [0]:
plt.figure(figsize=(10, 6))
plt.plot(data, color='red', label='actual data')
plt.plot(train_Plot_1, color='blue', linestyle="-", label='train data')
plt.plot(test_Plot_1, color='green', linestyle="-", label='test data')
plt.xlabel('Time [Days]', fontsize=15)
plt.ylabel('Stock Values', fontsize=15)
plt.title('Baseline Model Predictions on HGK20.CXM', fontsize=15)
plt.legend(fontsize=15)
plt.show()

In [0]:
plt.figure(figsize=(10, 6))
plt.plot(Y_test, color='red', label='actual data')
plt.plot(test_Predict1, color='blue', label='test data')
plt.xlabel('Time [Days]', fontsize=15)
plt.ylabel('Stock Values', fontsize=15)
plt.title('Real values vs Prediction values', fontsize=15)
plt.legend(fontsize=15)
plt.show()

## Retro-causal (stateful = true) Model

In [0]:
model_stateful1 = tf.keras.Sequential([
tf.keras.layers.LSTM(64,  batch_input_shape=(1, 12, 1), go_backwards=True, stateful=True),
tf.keras.layers.Dense(1)])

In [0]:
model_stateful1.compile(loss = 'mean_squared_error', optimizer='adam')
model_stateful1.summary()

In [0]:
history = model_stateful1.fit(X_train, y_train, epochs=100, batch_size=1, verbose = 1, validation_data=(X_test, Y_test), shuffle=False)
model_stateful1.reset_states()

In [0]:
batch_size=1
train_Score = model_stateful1.evaluate(X_train, y_train, batch_size=batch_size, verbose=1)
model_stateful1.reset_states()
train_Score = math.sqrt(train_Score)
train_Score = sc.inverse_transform(np.array([[train_Score]]))

In [0]:
test_Score = model_stateful1.evaluate(X_test, Y_test, batch_size=1, verbose=1)
model_stateful1.reset_states()
test_Score = math.sqrt(test_Score)
test_Score = sc.inverse_transform(np.array([[test_Score]]))

In [0]:
print('This is the train_Score:',train_Score)
print('This is the test_Score:', test_Score)

In [0]:
train_Predicts = model_stateful1.predict(X_train, batch_size=1)
test_Predicts = model_stateful1.predict(X_test, batch_size=1)

In [0]:
mse = mean_squared_error(Y_test, test_Predicts)
RMSE = np.sqrt(mse)
mae = mean_absolute_error(Y_test, test_Predicts)

print('The Mean Squared Error is: ', mse)
print('The Root Mean Squared Error is :', RMSE)
print('The Mean Absolute Error is ', mae)

In [0]:
test_Predicts

In [0]:
# Shifting train predictions for plotting
train_Plots = np.empty_like(data)
train_Plots[:, :] = np.nan
train_Plots[look_back:len(train_Predicts)+look_back, :] = train_Predicts

In [0]:
# Shifting test predictions for plotting
test_Plots = np.empty_like(data)
test_Plots[:, :] = np.nan
test_Plots[len(train_Predicts)+(look_back*2)+1:len(data)-1, :] = test_Predicts

In [0]:
plt.figure(figsize=(10, 6))
plt.plot(data, color='red', label='actual data')
plt.plot(train_Plots, color='blue', linestyle="-", label='train data')
plt.plot(test_Plots, color='green', linestyle="-", label='test data')
plt.xlabel('Time [Days]', fontsize=15)
plt.ylabel('Stock Values', fontsize=15)
plt.title('Baseline Model Predictions on HGK20.CXM', fontsize=15)
plt.legend(fontsize=15)
plt.show()

In [0]:
plt.figure(figsize=(10, 6))
plt.plot(Y_test, color='red', label='actual data')
plt.plot(test_Predicts, color='blue', label='test data')
plt.xlabel('Time [Days]', fontsize=15)
plt.ylabel('Stock Values', fontsize=15)
plt.title('Real values vs Prediction values', fontsize=15)
plt.legend(fontsize=15)
plt.show()

## Retro-causal (stateful = false) Model

In [0]:
model_stateless1 = tf.keras.Sequential([
tf.keras.layers.LSTM(64, input_shape=(12, 1), go_backwards=True, stateful=False),
tf.keras.layers.Dense(1)])

In [0]:
model_stateless1.compile(loss = 'mean_squared_error', optimizer='adam')
model_stateless1.summary()

In [0]:
history = model_stateless1.fit(X_train, y_train, epochs=100, batch_size=1, verbose = 1, validation_data=(X_test, Y_test), shuffle=False)

In [0]:
batch_size=1
train_Score = model_stateless1.evaluate(X_train, y_train, batch_size=batch_size, verbose=1)
model_stateless1.reset_states()
train_Score = math.sqrt(train_Score)
train_Score = sc.inverse_transform(np.array([[train_Score]]))

In [0]:
test_Score = model_stateless1.evaluate(X_test, Y_test, batch_size=batch_size, verbose=1)
model_stateless1.reset_states()
test_Score = math.sqrt(test_Score)
test_Score = sc.inverse_transform(np.array([[test_Score]]))

In [0]:
print('This is the train_Score:',train_Score)
print('This is the test_Score:', test_Score)

In [0]:
# generate predictions for training and testing
train_Predict_2 = model_stateless1.predict(X_train, batch_size=1)
test_Predict_2 = model_stateless1.predict(X_test, batch_size=1)

In [0]:
mse = mean_squared_error(Y_test, test_Predict_2)
RMSE = np.sqrt(mse)
mae = mean_absolute_error(Y_test, test_Predict_2)

print('The Mean Squared Error is: ', mse)
print
print('The Root Mean Squared Error is :', RMSE)
print
print('The Mean Absolute Error is ', mae)

In [0]:
test_Predict_2

In [0]:
# Shifting train predictions for plotting
train_Plot_2 = np.empty_like(data)
train_Plot_2[:, :] = np.nan
train_Plot_2[look_back:len(train_Predict_2)+look_back, :] = train_Predict_2

In [0]:
# Shifting test predictions for plotting
test_Plot_2 = np.empty_like(data)
test_Plot_2[:, :] = np.nan
test_Plot_2[len(train_Predict_2)+(look_back*2)+1:len(data)-1, :] = test_Predict_2

In [0]:
plt.figure(figsize=(10, 6))
plt.plot(data, color='red', label='actual data')
plt.plot(train_Plot_2, color='blue', linestyle="-", label='train data')
plt.plot(test_Plot_2, color='green', linestyle="-", label='test data')
plt.xlabel('Time [Days]', fontsize=15)
plt.ylabel('Stock Values', fontsize=15)
plt.title('Baseline Model Predictions on HGK20.CXM', fontsize=15)
plt.legend(fontsize=15)
plt.show()

In [0]:
plt.figure(figsize=(10, 6))
plt.plot(Y_test, color='red', label='actual data')
plt.plot(test_Predict_2, color='blue', label='test data')
plt.xlabel('Time [Days]', fontsize=15)
plt.ylabel('Stock Values', fontsize=15)
plt.title('Real values vs Prediction values', fontsize=15)
plt.legend(fontsize=15)
plt.show()

## Bidirectional LSTM Model

In [0]:
from keras.layers import Bidirectional
model_bi = tf.keras.Sequential()
model_bi.add(layers.Bidirectional(layers.LSTM(64, input_shape=(12,1)))),
model_bi.add(layers.Dense(1))

In [0]:
model_bi.compile(loss = 'mean_squared_error', optimizer='adam')

In [0]:
history = model_bi.fit(X_train, y_train, epochs=100, batch_size=1, verbose = 1, validation_data=(X_test, Y_test), shuffle=False)
model_bi.reset_states()

In [0]:
model_bi.summary()

In [0]:
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], color='blue', label='train loss')
plt.title('Model Loss')
plt.ylabel('Mean Squared Error')
plt.xlabel('Epoch')
plt.show()

In [0]:
batch_size=1
train_Score = model_bi.evaluate(X_train, y_train, batch_size=batch_size, verbose=0)
model_bi.reset_states()
train_Score = math.sqrt(train_Score)
train_Score = sc.inverse_transform(np.array([[train_Score]]))

test_Score = model_bi.evaluate(X_test, Y_test, batch_size=batch_size, verbose=0)
model_bi.reset_states()
test_Score = math.sqrt(test_Score)
test_Score = sc.inverse_transform(np.array([[test_Score]]))

In [0]:
print('This is the train_Score:',train_Score)
print('This is the test_Score:', test_Score)

In [0]:
# generate predictions for training and testing
train_Predict_bi = model_bi.predict(X_train, batch_size=1)
test_Predict_bi = model_bi.predict(X_test, batch_size=1)

In [0]:
mse = mean_squared_error(Y_test, test_Predict_bi)
RMSE = np.sqrt(mse)
mae = mean_absolute_error(Y_test, test_Predict_bi)

print('The Mean Squared Error is: ', mse)
print
print('The Root Mean Squared Error is :', RMSE)
print
print('The Mean Absolute Error is ', mae)

In [0]:
test_Predict_bi

In [0]:
# Shifting train predictions for plotting
train_Plot_bi = np.empty_like(data)
train_Plot_bi[:, :] = np.nan
train_Plot_bi[look_back:len(train_Predict_bi)+look_back, :] = train_Predict_bi

In [0]:
# Shifting test predictions for plotting
test_Plot_bi = np.empty_like(data)
test_Plot_bi[:, :] = np.nan
test_Plot_bi[len(train_Predict_bi)+(look_back*2)+1:len(data)-1, :] = test_Predict_bi

In [0]:
plt.figure(figsize=(10, 6))
plt.plot(data, color='red', label='actual data')
plt.plot(train_Plot_bi, color='blue', linestyle="-", label='train data')
plt.plot(test_Plot_bi, color='green', linestyle="-", label='test data')
plt.xlabel('Time [Days]', fontsize=15)
plt.ylabel('Stock Values', fontsize=15)
plt.title('Baseline Model Predictions on HGK20.CXM', fontsize=15)
plt.legend(fontsize=15)
plt.show()

In [0]:
plt.figure(figsize=(10, 6))
plt.plot(Y_test, color='red', label='actual data')
plt.plot(test_Predict_bi, color='blue', label='test data')
plt.xlabel('Time [Days]', fontsize=15)
plt.ylabel('Stock Values', fontsize=15)
plt.title('Real values vs Prediction values', fontsize=15)
plt.legend(fontsize=15)
plt.show()

In [0]:
plt.figure(figsize=(10, 6))
plt.plot(Y_test, color='red', label='actual data')
plt.plot(test_Predict, color='blue', linestyle="-", label='stateful=true')
plt.plot(test_Predict1, color='yellow', linestyle="-", label='stateful=false')
plt.plot(test_Predicts, color='orange', linestyle="-", label='stateful1=true')
plt.plot(test_Predict_2, color='violet', linestyle="-", label='stateful1=false')
plt.plot(test_Predict_bi, color='green', linestyle="-", label='bidirectional')
plt.xlabel('Time [Days]', fontsize=15)
plt.ylabel('Stock Values', fontsize=15)
plt.title('Real values vs Prediction values', fontsize=15)
plt.legend(fontsize=15)
plt.show()